In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('/Users/hzm/Desktop/YangTeng/code/venv/code/00.module')
import Proxy
import UA
import time

# = = = = = = = = = = = = = = = = = =

import pandas as pd
from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=' + Proxy.get_Proxy_selenium())
option.add_argument('user-agent=' + UA.get_UA())
option.page_load_strategy = 'normal'

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./kit.xlsx', header=0).fillna('')
list_info = df_menu['Kit_Info'].to_list()
list_info = list(set(list_info))

print('总数量：' + str(len(list_info)))
print()

df = pd.DataFrame()

# = = = = = = = = = = = = = = = = = =

work = Queue()
for info in list_info:
    work.put_nowait(info)

def crawler():
    global df
    
    while not work.empty():
        info = work.get_nowait()
        
        # = = = = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            if info == None or info == '':
                status = 'none'
                break
            
            try:
                browser = Chrome(service=Service('../../../chromedriver'), options=option)
                browser.maximize_window()
                
                # = = = = = = = = = = = = = = = = = =

                browser.get(info)
                
                time.sleep(3)
                
                # = = = = = = = = = = = = = = = = = =
                
                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//section[contains(@aria-label, "Warranty Information")]')))
                oe = browser.find_elements(by=By.XPATH, value='//table[contains(@class, "bdytext")]/descendant::section[contains(@aria-label, "Alternate/OEM Part Number(s)")]')

                if len(oe) == 0:
                    oe = ''
                else:
                    oe = oe[0].text.split('OEM / Interchange Numbers: ')[1].replace(', ', ';')

                # = = = = = = = = = = = = = = = = = =
                
                src = browser.find_elements(by=By.XPATH, value='//img[contains(@class, "listing-inline-image") and contains(@class, "listing-inline-image-moreinfo")]')

                if len(src) == 0:
                    src = ''
                else:
                    src = src[0].get_attribute('src')
                
                # = = = = = = = = = = = = = = = = = =
                
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Kit_Info': info,
                                         'Kit_OE': oe,
                                         'Kit_Src': src}])

                browser.quit()
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(3)
                continue
                
        # = = = = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Kit_Info': info}])
        elif status == 'none':
            df_temp = pd.DataFrame([{'status': 'none',
                                     'Kit_Info': info}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        if info != None and info != '':
            print(info +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(25):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('输出中...')
df.to_excel('./kit_oe.xlsx', index=False)
print()
print('搞定！')

总数量：151

https://www.rockauto.com/en/moreinfo.php?pk=8307544&pt=5728  <->  [ok] - 剩余数量：125
https://www.rockauto.com/en/moreinfo.php?pk=8921468&pt=5723  <->  [ok] - 剩余数量：124
https://www.rockauto.com/en/moreinfo.php?pk=10110028&pt=5736  <->  [ok] - 剩余数量：123
https://www.rockauto.com/en/moreinfo.php?pk=8307552&pt=5728  <->  [ok] - 剩余数量：122
https://www.rockauto.com/en/moreinfo.php?pk=9659108&pt=5736  <->  [ok] - 剩余数量：121
https://www.rockauto.com/en/moreinfo.php?pk=8306448&pt=5724  <->  [ok] - 剩余数量：120
https://www.rockauto.com/en/moreinfo.php?pk=8921276&pt=11859  <->  [ok] - 剩余数量：119
https://www.rockauto.com/en/moreinfo.php?pk=9659224&pt=5723  <->  [ok] - 剩余数量：118
https://www.rockauto.com/en/moreinfo.php?pk=8921472&pt=5589  <->  [ok] - 剩余数量：117
https://www.rockauto.com/en/moreinfo.php?pk=8313560&pt=10465  <->  [ok] - 剩余数量：116
https://www.rockauto.com/en/moreinfo.php?pk=9659100&pt=5728  <->  [ok] - 剩余数量：115
https://www.rockauto.com/en/moreinfo.php?pk=8313564&pt=5722  <->  [ok] - 剩余数量：114
http

KeyboardInterrupt
2023-08-23T01:17:44Z


KeyboardInterrupt: 